In [ ]:
import pandas as pd
book=pd.read_csv('../input/toy-dataset/toy_dataset.csv')
book.head()

In [ ]:
book.nunique()

In [ ]:
book.info()

In [ ]:
book.skew()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, axs = plt.subplots(1, 2, figsize=(10,5))
axs = axs.flatten()
sns.boxplot(y=book['Income'],ax=axs[0])
sns.boxplot(y=book['Age'],ax=axs[1])
plt.show()

In [ ]:
!pip install pandas-profiling

In [ ]:
from pandas_profiling import ProfileReport
ProfileReport(book, title="EDA Report")

In [ ]:
!pip install pycaret

In [ ]:
data = book.sample(frac=0.8, random_state=42)
data_unseen = book.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))

In [ ]:
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
round(book.Illness.value_counts()*100/len(book),2)

In [ ]:
round(book.City.value_counts()*100/len(book),2)

In [ ]:
from pycaret.classification import *

In [ ]:
toy= setup(data = data, target = 'Illness', session_id=42,
                  normalize = True, 
                  transformation = True, 
                  log_experiment = True,
                  handle_unknown_categorical = True, 
                  unknown_categorical_method = 'most_frequent',
                  remove_multicollinearity = True, #rop one of the two features that are highly correlated with each other
                  ignore_low_variance = True,#all categorical features with statistically insignificant variances are removed from the dataset.
                  combine_rare_levels = True,# all levels in categorical features below the threshold defined in rare_level_threshold param are combined together as a single level
                  numeric_imputation='median',
                  categorical_imputation='mode',
           ignore_features=['Number'],
           categorical_features = ['City','Gender'],
           fix_imbalance = True,
          normalize_method = 'robust',train_size = 0.8
          )

In [ ]:
rf = create_model('rf')

In [ ]:
print(rf)

In [ ]:
tuned_rf = tune_model(rf,optimize = 'MCC')

In [ ]:
plot_model(tuned_rf, plot = 'auc')

In [ ]:
plot_model(tuned_rf, plot = 'pr')

In [ ]:
plot_model(tuned_rf, plot = 'confusion_matrix')

In [ ]:
evaluate_model(tuned_rf)

In [ ]:
predict_model(tuned_rf)

In [ ]:
unseen_predictions = predict_model(tuned_rf, data=data_unseen)
unseen_predictions.head()

In [ ]:
print("Confidence Score :   {}".format(round(unseen_predictions.Score.mean(),2)))#Confidence Score